In [2]:
import pandas as pd
from datetime import datetime as dt
import os

path = os.path.join(os.getcwd() + "/Output")

if not (os.path.exists(path)):
    os.makedirs(path)
csv_files = []
for file in os.listdir(os.getcwd()):
    if file.endswith(".csv"):
        csv_files.append(file)

for files in csv_files:
    df2 = pd.read_csv(files)
    df3 = pd.read_csv('Source/Last_date.csv')
    for col in df3.columns[2:]:
        df3[col].values[:] = 0  
        
    ldt = pd.to_datetime(df2.iloc[-1]["Date"])

    df3["Date"] = ldt

    df3['Date'] = pd.to_datetime(df3['Date'])
    df3['Time'] = pd.to_datetime(df3['Time'],format="%H:%M:%S").dt.time

    df2['Date'] = pd.to_datetime(df2['Date'])
    df2['Time'] = pd.to_datetime(df2['Time'],format="%H:%M:%S").dt.time

    df3["DT"] = df3['Date'].astype(str) + ' ' + df3['Time'].astype(str)
    df2["DT"] = df2['Date'].astype(str) + ' ' + df2['Time'].astype(str)

    r1 = df2[(pd.to_datetime(df2['DT'])).isin(pd.to_datetime(df3['DT']))]

    r2 = df3[~(pd.to_datetime(df3['DT'])).isin(pd.to_datetime(df2['DT']))]


    df = pd.concat([r2,r1],axis=0,ignore_index = True)

    df = df.sort_values(by="DT",ignore_index = True)

    r3 = df2[~(pd.to_datetime(df2['DT'])).isin(pd.to_datetime(df['DT']))]

    df = r3.append(df, ignore_index=True)
    df['Day'] = 0
    for days in range(len(df)):
        if (df.loc[days ,"Date"].weekday() == 0):
            df.loc[days,'Day'] = 2
        elif (df.loc[days,"Date"].weekday() == 1):
            df.loc[days,"Day"] = 3
        elif (df.loc[days,"Date"].weekday() == 2):
            df.loc[days,"Day"] = 4
        elif (df.loc[days,"Date"].weekday() == 3):
            df.loc[days,"Day"] = 5
        else :
            df.loc[days,"Day"] = 1

    for i in range(len(df) - 1):
        if df.loc[i,'Day'] > df.loc[i+1,'Day']:
            df.loc[i+1,'Day'] = -1

    for i in range(len(df) - 1):    
        if (df.loc[i + 1, 'Day'] != -1) & (df.loc[i + 1, "Open"] == 0) &(df.loc[i+1,"Date"] == ldt):
            df.loc[i+1, "Open"] = df.loc[i, "Close"] 
            df.loc[i + 1, 'High'] = df.loc[i, "Close"]
            df.loc[i + 1, 'Low'] = df.loc[i, "Close"]
            df.loc[i + 1, 'Close'] = df.loc[i, "Close"]
            df.loc[i+1, "Open Interest"] = df.loc[i, "Open Interest"]
    df = df.drop(['DT', 'Day'], axis=1)

    df.to_csv(f"Output/{files}", index=False)

KeyboardInterrupt: 